##### Find cars on an image

In [1]:
import numpy as np
import pickle
import cv2
import glob
import os
import time
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import pandas as pd
from moviepy.editor import VideoFileClip
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.ndimage.measurements import label

##### Data collection

In [2]:
car_images = glob.glob('./training_data/vehicles/*/*/*.png')
noncar_images = glob.glob('./training_data/non_vehicles/*/*/*.png')

##### Set hog parameters, result of hog parameter exploration step

In [3]:
hog_params = {}
hog_params['cspace'] = 'YUV'
hog_params['orient'] = 10
hog_params['pix_per_cell'] = 16
hog_params['cell_per_block'] = 2
hog_params['hog_channel'] = "ALL"

##### Train using SVC after getting hog features

In [4]:
from vehicle_detection import classifier
print('Training with hog features...........')
g_svc, hog_svc_data = classifier.train_hog_features(car_images, noncar_images, hog_params)
print('Training done...........')

Training with hog features...........
56.36 Seconds to extract HOG features...
Using: 10 orientations 16 pixels per cell and 2 cells per block
Feature vector length: 1080
1.38 Seconds to train SVC...
Test Accuracy of SVC =  0.9837
My SVC predicts:  [ 1.  1.  0.  0.  0.  0.  0.  0.  1.  0.]
For these 10 labels:  [ 1.  1.  0.  0.  0.  0.  0.  0.  1.  0.]
0.02804 Seconds to predict 10 labels with SVC
Training done...........


##### Begin sliding window search

In [5]:
from vehicle_detection import sliding_window

##### Set parameters for window, final parameters chosen after trial and study done in vehicle_detection_sliding_window.ipynb

In [6]:
g_sliding_window_params = [[400,450,1],[425,475,1],[450,475,1],
                           [400,475,1.5],[425,500,1.5],[400,500,1.5],[400,525,1.5],
                           [400,600,2],[400,650,2], [450,650,2],
                           [400,650,3], [400,700,3]]
g_threshold = 10

g_colorspace = hog_params['cspace']
g_hog_channel = hog_params['hog_channel']
g_orient = hog_params['orient']
g_pix_per_cell = hog_params['pix_per_cell']
g_cell_per_block = hog_params['cell_per_block']

##### Function to perform  sliding window search on provided set of window parameters, chosen after trials performed

In [7]:
def car_location_exploration(img, g_sliding_window_params):
    car_windows = []
    
    for params in g_sliding_window_params:
        ystart = params[0]
        ystop = params[1]
        scale = params[2]
            
        #Detect cars
        car_windows.append(sliding_window.detect_cars(img, ystart, ystop, scale, g_svc, g_colorspace, g_hog_channel, 
                  g_orient, g_pix_per_cell, g_cell_per_block))
    return car_windows

#### Define a class to remember the positions of each car detection

In [8]:
class carPosition():
    def __init__(self, n):
        # Number of past frames to be remembered
        self.n_frame = n
        # car position values of the last n detections
        self.recent_car_positions = [] 
        
        
    def update_detected_car_data(self, car_positions):
        '''
        Update the car positions of current frame.
        '''
        self.recent_car_positions.append(car_positions)
        if len(self.recent_car_positions) > self.n_frame:
            #Keep latest n_frames
            self.recent_car_positions = self.recent_car_positions[len(self.recent_car_positions)-self.n_frame:]

In [9]:
def detect_car_pipeline_video(test_image):
    '''
    Function to process car detection on a video, which makes use of the carPosition class to remmeber
    detections from past frames.
    '''
    car_windows = car_location_exploration(test_image, g_sliding_window_params)
    car_positions = [item for sublist in car_windows for item in sublist] 
    
    if len(car_positions)>0:
        car_pos.update_detected_car_data(car_positions)
    
    # Test out the heatmap
    heatmap_img = np.zeros_like(test_image[:,:,0])
    for positions in car_pos.recent_car_positions:
        heatmap_img = sliding_window.add_heat(heatmap_img, positions)
   
    thresholded_img = sliding_window.apply_threshold(heatmap_img, g_threshold + len(car_pos.recent_car_positions)//2)
   
    labels = label(thresholded_img)
    # Draw bounding boxes on a copy of the image
    draw_img = sliding_window.draw_labeled_bboxes(np.copy(test_image), labels)
 
    return draw_img

#### Test on project video

In [10]:
car_pos = carPosition(20)
from moviepy.editor import VideoFileClip
test_out_file = 'project_video_out.mp4'
clip_test = VideoFileClip('project_video.mp4')
clip_test_out = clip_test.fl_image(detect_car_pipeline_video)
%time clip_test_out.write_videofile(test_out_file, audio=False)

[MoviePy] >>>> Building video project_video_out.mp4
[MoviePy] Writing video project_video_out.mp4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [10:18<00:00,  2.19it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_out.mp4 

Wall time: 10min 20s
